In [8]:
pip install langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [39]:
import os
from openai import OpenAI
from pydantic import BaseModel

os.environ["OPENAI_API_KEY"] = ""

In [41]:
class SenseOfControl(BaseModel):
    cost_of_living: int
    safety: int
    influence_and_contribution: int
    
class HealthEquity(BaseModel):
    housing_standard: int
    air_noise_light: int
    food_choice: int
    
class ConnectionToNature(BaseModel):
    housing_standard: int
    air_noise_light: int
    food_choice: int
    
class SenseOfWonder(BaseModel):
    distinctive_design_and_culture: int
    play_and_recreation: int
    
class GettingAround(BaseModel):
    walking_and_cycling: int
    public_transport: int
    car: int

class ConnectedCommunities(BaseModel):
    belonging: int
    local_business_and_jobs: int

class QualityOfLifeIndexes(BaseModel):
    a_sense_of_control: SenseOfControl
    a_sense_of_wonder: SenseOfWonder
    connected_communities: ConnectedCommunities
    connection_to_nature: ConnectionToNature
    getting_around: GettingAround
    health_equity: HealthEquity

In [48]:
def get_openai_response(prompt: str, model: str = "gpt-4o-mini", max_tokens: int = 500, temperature: float = 0.2) -> str:
    """
    Connects to the OpenAI API and submits a prompt, returning the response.

    Parameters:
    - prompt (str): The text prompt to submit to OpenAI.
    - model (str): The model to use, e.g., "gpt-4o". Defaults to "gpt-4".
    - max_tokens (int): The maximum number of tokens in the response. Defaults to 500.
    - temperature (float): The creativity level of the response. Defaults to 0.7.

    Returns:
    - str: The response from the OpenAI API.
    """
    
    try:
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

        chat_completion = client.beta.chat.completions.parse(
            model=model,
            messages=[
                {"role": "system", "content": "You are an analyst assistant which helps generate scores between 0 to 100 on various aspects of quality of life based on questions around quality of life indexes and user responses."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature,
            response_format=QualityOfLifeIndexes
        )
        
        # Extracting the text response
        print(chat_completion)
        return chat_completion.choices[0].message

    except Exception as e:
        return f"An error occurred: {e}"

In [49]:
def create_prompt_user_response_matrix(user_response: dict) -> str:
    """
    Creates the prompt for converting user responses to qol matrix.
    Parameters: 
    - user_response (dict): The response from the OpenAI API.
    Returns:
    - prompt (str): The prompt created for extracting qol matrix from user response.
    """
    prompt = f"""
        We are designing a tool which gathers user responses on questions around quality of life aspects for housing and recommends them areas to live in.
        We have decided to use 6 indexes of measuring quality of life with each one of them having multiple factors. Alongside these indexes to understand user demographics we also get user information like number of adults, children and pets in the family, whether the user has a preferred city or county where the user would like to move, and whether the user wants to live in an urban, semi-urban or rural environment. We also get additional context from user which captures important information like must haves for user and whether there is a deal breaker or non negotiable item for user
        
        Here is the information for the indexes
        First index is sense of control which has 4 factors in it cost of living, essential services, safety and influence and contribution
        Second index is a sense of wonder which has 2 factors play & recreation and distinctive design & culture
        Third index is connected communities which has 2 factors local business and jobs and belonging
        Fourth index is connection to nature which has 3 factors biodiversity, climate resilience and adaptation, and green and blue spaces
        Fifth index is getting around which has 3 factors cars, public transport and walking and cycling
        Sixth index is health equity which has 3 factors air noise light, food choice, and housing standards
        
        Here are the questions and responses gathered from a user on these indexes and factors:
        
        Demographics Information
        Question: {user_response['demographics']['number_of_family_members']['question']}
        User response: {user_response['demographics']['number_of_family_members']['response']}
        
        Question: {user_response['demographics']['city_or_county_preference']['question']}
        User response: {user_response['demographics']['city_or_county_preference']['response']}
        
        Question: {user_response['demographics']['urban_or_rural_preference']['question']}
        User response: {user_response['demographics']['urban_or_rural_preference']['response']}
        
        Index name: A sense of control
        Factors: cost of living
        Question: {user_response['a_sense_of_control']['cost_of_living']['question']}
        User response: {user_response['a_sense_of_control']['cost_of_living']['response']}
        
        Index name: A sense of control
        Factors: essential services
        Question: {user_response['a_sense_of_control']['essential_services']['question']}
        User response: {user_response['a_sense_of_control']['essential_services']['response']}
        
        Index name: A sense of control
        Factors: safety
        Question: {user_response['a_sense_of_control']['safety']['question']}
        User response: {user_response['a_sense_of_control']['safety']['response']}
        
        Index name: A sense of control
        Factors: influence and contribution
        Question: {user_response['a_sense_of_control']['influence_and_contribution']['question']}
        User response: {user_response['a_sense_of_control']['influence_and_contribution']['response']}
        
        Index name: A sense of wonder
        Factors: distinctive design and culture
        Question: {user_response['a_sense_of_wonder']['distinctive_design_and_culture']['question']}
        User response: {user_response['a_sense_of_wonder']['distinctive_design_and_culture']['response']}
        
        Index name: A sense of wonder
        Factors: play and recreation
        Question: {user_response['a_sense_of_wonder']['play_and_recreation']['question']}
        User response: {user_response['a_sense_of_wonder']['play_and_recreation']['response']}
        
        Index name: Connected Communities
        Factors: local business & jobs and belonging
        Question: {user_response['connected_communities']['local_business_and_jobs']['question']}
        User response: {user_response['connected_communities']['local_business_and_jobs']['response']}
        
        Index name: Connection to nature
        Factors: biodiversity, climate resilience & adaptation and green & blue spaces
        Question: {user_response['connection_to_nature']['biodiversity']['question']}
        User response: {user_response['connection_to_nature']['biodiversity']['response']}
        
        Index name: Getting Around
        Factors: car
        Question: {user_response['getting_around']['car']['question']}
        User response: {user_response['getting_around']['car']['response']}
        
        Index name: Getting Around
        Factors: public transport
        Question: {user_response['getting_around']['public_transport']['question']}
        User response: {user_response['getting_around']['public_transport']['response']}
        
        Index name: Getting Around
        Factors: walking and cycling
        Question: {user_response['getting_around']['walking_cycling']['question']}
        User response: {user_response['getting_around']['walking_cycling']['response']}
        
        Index name: Health Equity
        Factors: air, noise and light
        Question: {user_response['health_equity']['air_noise_light']['question']}
        User response: {user_response['health_equity']['air_noise_light']['response']}
        
        Index name: Health Equity
        Factors: food choice
        Question: {user_response['health_equity']['food_choice']['question']}
        User response: {user_response['health_equity']['food_choice']['response']}
        
        Index name: Health Equity
        Factors: housing standard
        Question: {user_response['health_equity']['housing_standard']['question']}
        User response: {user_response['health_equity']['housing_standard']['response']}
        
        Index name: Health Equity
        Factors: allergies
        Question: {user_response['health_equity']['allergies']['question']}
        User response: {user_response['health_equity']['allergies']['response']}
        
        Additional Context
        Question: {user_response['additional_context']['must_have']['question']}
        User response: {user_response['additional_context']['must_have']['response']}
        
        Here is the task, you have to create a JSON object which contains index, factor and score of 0 to 100 for each factor based on the questions and user response from above data
        Here is an example output JSON, please use just the JSON structure for creating the output and do not use the numbers in JSON for your calculation
        
        {{
            "a_sense_of_control": {{
                "cost_of_living": 65,
                "safety": 78,
                "influence_and_contribution": 55
            }},
            "health_equity": {{
                "housing_standard": 58,
                "air_noise_light": 82,
                "food_choice": 62
            }},
            "connection_to_nature": {{
                "green_and_blue_spaces": 75,
                "biodiversity": 68,
                "climate_resilience_and_adaptation": 72
            }},
            "a_sense_of_wonder": {{
                "distinctive_design_and_culture": 60,
                "play_and_recreation": 70
            }},
            "getting_around": {{
                "walking_and_cycling": 52,
                "public_transport": 48,
                "car": 68
            }},
            "connected_communities": {{
                "belonging": 72,
                "local_business_and_jobs": 58
            }},
        }}
    """
    return prompt

questions = [
    "May I ask how many adults, children, and pets will be joining you in your new home?",
    "Would you like to narrow down your search to a specific city or county, or are you open to exploring different areas?",
    "What type of area are you most interested in? Are you looking for an urban environment, a metropolitan area, a semi-urban setting, a small town, a village, or perhaps a micropolitan area?",
    "What would be your ideal monthly budget for living expenses, including housing, utilities, and transportation?",
    "How important is it for you to have essential services like hospitals, schools, and grocery stores nearby?",
    "How significant is safety for you when choosing a neighborhood, particularly in terms of low crime rates and a strong community atmosphere?",
    "How often do you participate in community events or local meetings? Is this something you'd like to continue in your new area?",
    "How frequently do you plan to visit parks, recreational facilities, or engage in outdoor activities in your new neighborhood?",
    "How much do cultural events, such as festivals or museum visits, play a role in your lifestyle? How many would you like to attend each year?",
    "How important is it for you to live in a community with thriving local businesses, job opportunities, and a strong sense of belonging?",
    "How often would you like to spend time in green spaces, parks, or near natural water bodies in your new area?",
    "Do you plan to use a car for commuting? If so, how much time are you comfortable spending in the car each day?",
    "How often do you anticipate using public transport, and how important is it for you to have convenient access to it?",
    "How important is it for you to live in an area that is pedestrian-friendly and has good biking infrastructure? How often do you plan to walk or bike for your daily activities?",
    "How important is it to you to live in a place with good air and water quality, as well as minimal noise and light pollution?",
    "Is access to healthy food options and nearby grocery stores a significant factor in your decision-making process?",
    "How crucial is it for you to live in an area with high housing standards, including well-maintained properties and infrastructure?",
    "Do you or any family members have allergies (such as pollen) or specific health conditions that should be considered?",
    "Is there anything specific that’s especially important to you in a new home, something that would be non-negotiable or a deal-breaker?"
]


response = [
    "I have a family of 2 adults",
    "I am open to moving anywhere in Ireland",
    "I prefer semi-urban environments",
    "My monthly budget is around 3000 euros per month",
    "I would like to have a hospital and grocery stores in vicinity of 5 kilometers",
    "Safety is very important to me, I would like to live in a area with low crime rates",
    "I attend around 3 such meetings in a year",
    "I attend 7 such events in a year",
    "I play Tennis about 3 times a year and visit gym and parks frequently",
    "A sense of belonging and job opportunities are important to me, thriving local businesses is a plus to have",
    "I visit beaches about once a month and other spaces about 3 times in a month",
    "I have a car and use it for most of my commute, I spend around 30 minutes driving daily",
    "I would also like to have good connectivity by public transport in the area where I live, I plan to use public transport atleast 2 times a week",
    "Biking and pedestrian infra would be nice to have",
    "Air and water quality are important to me",
    "Healthy food options are nice to have and I will need grocery stores in the 5 km radius of my house",
    "I would like to live in an area which has good-looking houses and well maintained infrastructure",
    "No allergies or health conditions in our family",
    "The house should be affordable and should have good public transport connectivity"
]


user_response_dict = {
    "demographics" : {
        "number_of_family_members": {
            "question" : questions[0],
            "response" : response[0]
        },
        "city_or_county_preference": {
            "question" : questions[1],
            "response" : response[1]
        },
        "urban_or_rural_preference": {
            "question" : questions[2],
            "response" : response[2]
        },
    },
    "a_sense_of_control" : {
        "cost_of_living": {
            "question" : questions[3],
            "response" : response[3]
        },
        "essential_services": {
            "question" : questions[4],
            "response" : response[4]
        },
        "safety": {
            "question" : questions[5],
            "response" : response[5]
        },
        "influence_and_contribution": {
            "question" : questions[6],
            "response" : response[6]
        }
    },
    "a_sense_of_wonder" : {
        "play_and_recreation": {
            "question" : questions[7],
            "response" : response[7]
        },
        "distinctive_design_and_culture": {
            "question" : questions[8],
            "response" : response[8]
        }
    },
    "connected_communities" : {
        "local_business_and_jobs": {
            "question" : questions[9],
            "response" : response[9]
        }
    },
    "connection_to_nature" : {
        "biodiversity": {
            "question" : questions[10],
            "response" : response[10]
        }
    },
    "getting_around" : {
        "car": {
            "question" : questions[11],
            "response" : response[11]
        },
        "public_transport": {
            "question" : questions[12],
            "response" : response[12]
        },
        "walking_cycling": {
            "question" : questions[13],
            "response" : response[13]
        }
    },
    "health_equity" : {
        "air_noise_light": {
            "question" : questions[14],
            "response" : response[14]
        },
        "food_choice": {
            "question" : questions[15],
            "response" : response[15]
        },
        "housing_standard": {
            "question" : questions[16],
            "response" : response[16]
        },
        "allergies": {
            "question" : questions[17],
            "response" : response[17]
        }
    },
    "additional_context" : {
        "must_have": {
            "question" : questions[18],
            "response" : response[18]
        }
    }
}

prompt = create_prompt_user_response_matrix(user_response_dict)
prompt

'\n        We are designing a tool which gathers user responses on questions around quality of life aspects for housing and recommends them areas to live in.\n        We have decided to use 6 indexes of measuring quality of life with each one of them having multiple factors. Alongside these indexes to understand user demographics we also get user information like number of adults, children and pets in the family, whether the user has a preferred city or county where the user would like to move, and whether the user wants to live in an urban, semi-urban or rural environment. We also get additional context from user which captures important information like must haves for user and whether there is a deal breaker or non negotiable item for user\n        \n        Here is the information for the indexes\n        First index is sense of control which has 4 factors in it cost of living, essential services, safety and influence and contribution\n        Second index is a sense of wonder which

In [50]:
prompt = create_prompt_user_response_matrix(user_response_dict)
response = get_openai_response(prompt)
response

ParsedChatCompletion[QualityOfLifeIndexes](id='chatcmpl-A1i5yrp5YEe3cWSS0XiiiPq7badO4', choices=[ParsedChoice[QualityOfLifeIndexes](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[QualityOfLifeIndexes](content='{\n    "a_sense_of_control": {\n        "cost_of_living": 70,\n        "safety": 85,\n        "influence_and_contribution": 60\n    },\n    "a_sense_of_wonder": {\n        "distinctive_design_and_culture": 65,\n        "play_and_recreation": 75\n    },\n    "connected_communities": {\n        "belonging": 80,\n        "local_business_and_jobs": 70\n    },\n    "connection_to_nature": {\n        "housing_standard": 75,\n        "air_noise_light": 80,\n        "food_choice": 65\n    },\n    "getting_around": {\n        "walking_and_cycling": 55,\n        "public_transport": 70,\n        "car": 75\n    },\n    "health_equity": {\n        "housing_standard": 70,\n        "air_noise_light": 75,\n        "food_choice": 65\n    }\n}', refusal=None, rol

ParsedChatCompletionMessage[QualityOfLifeIndexes](content='{\n    "a_sense_of_control": {\n        "cost_of_living": 70,\n        "safety": 85,\n        "influence_and_contribution": 60\n    },\n    "a_sense_of_wonder": {\n        "distinctive_design_and_culture": 65,\n        "play_and_recreation": 75\n    },\n    "connected_communities": {\n        "belonging": 80,\n        "local_business_and_jobs": 70\n    },\n    "connection_to_nature": {\n        "housing_standard": 75,\n        "air_noise_light": 80,\n        "food_choice": 65\n    },\n    "getting_around": {\n        "walking_and_cycling": 55,\n        "public_transport": 70,\n        "car": 75\n    },\n    "health_equity": {\n        "housing_standard": 70,\n        "air_noise_light": 75,\n        "food_choice": 65\n    }\n}', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=QualityOfLifeIndexes(a_sense_of_control=SenseOfControl(cost_of_living=70, safety=85, influence_and_contribution=60), a_sense_of_w

In [51]:
response.parsed

QualityOfLifeIndexes(a_sense_of_control=SenseOfControl(cost_of_living=70, safety=85, influence_and_contribution=60), a_sense_of_wonder=SenseOfWonder(distinctive_design_and_culture=65, play_and_recreation=75), connected_communities=ConnectedCommunities(belonging=80, local_business_and_jobs=70), connection_to_nature=ConnectionToNature(housing_standard=75, air_noise_light=80, food_choice=65), getting_around=GettingAround(walking_and_cycling=55, public_transport=70, car=75), health_equity=HealthEquity(housing_standard=70, air_noise_light=75, food_choice=65))

In [ ]:
{
    "a_sense_of_control": {
        "cost_of_living": 65,
        "safety": 78,
        "influence_and_contribution": 55
    },
    "health_equity": {
        "housing_standard": 58,
        "air_noise_light": 82,
        "food_choice": 62
    },
    "connection_to_nature": {
        "green_and_blue_spaces": 75,
        "biodiversity": 68,
        "climate_resilience_and_adaptation": 72
    },
    "a_sense_of_wonder": {
        "distinctive_design_and_culture": 60,
        "play_and_recreation": 70
    },
    "getting_around": {
        "walking_and_cycling": 52,
        "public_transport": 48,
        "car": 68
    },
    "connected_communities": {
        "belonging": 72,
        "local_business_and_jobs": 58
    },
}